# Database Creation <a class="tocSkip">
Author: Stefan Roland Schwingenschlögl <br>
email: stefan.roland.schwingenschloegl@gmail.com <br>
github: github.com/stefan-schwingenschloegl <br>
___
*Projekt File No: 3 <br>*
    
In this notebook a Database on MS SQL Server Express will be created. In order to create this DB with python the library `pyodbc` will be used. At first for every static file from the Wiener Linien API a table will be created. This tables will be filled with the cleaned values from the notebook before. <br>
In the last step an additional table will be created. In this table all entries from the Live delays will be saved in this table.

In [1]:
#Libraries for handling operation system
import os

#General Libraries for data processing
import numpy as np
import pandas as pd

# Library for database connection and operations
import pyodbc

In [2]:
# set string to data folders
input_folder = "./input_data/"

In [3]:
#get list of all filenames in data

def get_all_filenames_in_data():
    file_list = []
    for (dirpath, dirnames, filenames) in os.walk(input_folder):
        file_list.extend(filenames)
        #break
    return file_list

In [4]:
#read all cleaned data with the given suffix

def read_all_files():
    file_list = get_all_filenames_in_data()
    data = {}
    data_gtfs = {}
    for file in file_list:
        if file.endswith('.txt'):
            #df = pd.read_csv('./input_data/' + file)
            data_gtfs[file] = pd.read_csv(input_folder + file)
        elif file.startswith('clean'):
            try:
                data[file] = pd.read_csv(input_folder + file, index_col = 0)
                data[file] = data[file].replace({np.nan: None})
            except pd.errors.ParserError:
                continue
        
    return data_gtfs, data

In [5]:
# call function for loading static files
[data_gtfs, data] = read_all_files()

# Database operations

In [6]:
# set properties for database
db_name = 'WienerLinienDB'
server = 'DESKTOP-JV1HTQR\SQLEXPRESS'
db_connection = False

## Create Database

In [7]:
# establish DB Connection
def db_connect(server, db_name):
    conn = pyodbc.connect("driver={SQL Server};server="+server+"; database="+db_name+"; trusted_connection=true")
    db_connection = True
    print(f"\nConnection with {server} sucessfull!\n"
          f"Current Database: {db_name}\n"
          f"DB Connection Status: {db_connection}")
    return conn

In [8]:
# close DB Connection
def close_connection(server, conn):
    conn.close()
    db_connection = False
    print(f"\nDB-Connection with Server {server} closed.\n"
          f"DB Connections Status: {db_connection}")

In [9]:
# create WL_delay Database if it does not exist
def create_DB(server, db_name):
    
    # establish connection to master database
    conn = db_connect(server=server, db_name='master')
    
    with conn:
        crs=conn.cursor()
        crs.execute(f"IF NOT EXISTS(SELECT * FROM sys.databases WHERE name = '{db_name}') CREATE DATABASE {db_name};")   
        print(f"{db_name} sucessfully created.")
    close_connection(server = server, conn=conn)

In [11]:
# create WienerLinienDB
create_DB(server, db_name)


Connection with DESKTOP-JV1HTQR\SQLEXPRESS sucessfull!
Current Database: master
DB Connection Status: True
WienerLinienDB sucessfully created.

DB-Connection with Server DESKTOP-JV1HTQR\SQLEXPRESS closed.
DB Connections Status: False


## Create Database Tables

In [12]:
# function to create table schema for static file
def create_table(table_name, sql_command, server, db_name):
    conn = db_connect(server=server, db_name=db_name)
    with conn:
        crs=conn.cursor()
        crs.execute(sql_command)
        print(f"\nTable '{table_name}' available in {db_name}")
    close_connection(server = server, conn=conn)

### Create and fill Haltepunkte

In [13]:
# create table haltepunkte
sql_command = '''IF NOT EXISTS(SELECT * FROM information_schema.tables
                 WHERE table_name='haltepunkte' AND table_schema='dbo') 
                 CREATE TABLE haltepunkte (
                        [StopID] int,
                        [DIVA] int,
                        [Municipality] varchar(50),
                        [MunicipalityID] int,
                        [StopText] varchar(100),
                        [Longitude] numeric(18,6),
                        [Latitude] numeric(18,6)
                        )
                    '''
create_table('haltepunkte', sql_command, server, db_name)


Connection with DESKTOP-JV1HTQR\SQLEXPRESS sucessfull!
Current Database: WienerLinienDB
DB Connection Status: True

Table 'haltepunkte' available in WienerLinienDB

DB-Connection with Server DESKTOP-JV1HTQR\SQLEXPRESS closed.
DB Connections Status: False


In [14]:
def fill_haltepunkte(sql_command, df, server, db_name):
    conn = db_connect(server=server, db_name=db_name)
    with conn:
        crs=conn.cursor()
        try:
            for index, row in df.iterrows():
                crs.execute(sql_command, row['StopID'],
                                         row['DIVA'],
                                         row['Municipality'],
                                         row['MunicipalityID'],
                                         row['StopText'],
                                         row['Longitude'],
                                         row['Latitude']
                                         )
            print('Table sucessfully filled!')
        except:
            print('Not Sucessfull!')
    close_connection(server = server, conn=conn)

In [15]:
sql_command= '''INSERT INTO [dbo].[haltepunkte]
           ([StopID]
           ,[DIVA]
           ,[Municipality]
           ,[MunicipalityID]
           ,[StopText]
           ,[Longitude]
           ,[Latitude])
     VALUES
           (?,
            ?,
            ?,
            ?,
            ?,
            ?,
            ?)
'''
fill_haltepunkte(sql_command, data['clean_haltepunkte'], server, db_name)


Connection with DESKTOP-JV1HTQR\SQLEXPRESS sucessfull!
Current Database: WienerLinienDB
DB Connection Status: True
Table sucessfully filled!

DB-Connection with Server DESKTOP-JV1HTQR\SQLEXPRESS closed.
DB Connections Status: False


### Create and fill Linien

In [16]:
# create table linien
sql_command = '''IF NOT EXISTS(SELECT * FROM information_schema.tables
                 WHERE table_name='linien' AND table_schema='dbo')
                 CREATE TABLE linien (
                     [LineID] int,
                     [LineText] varchar(50),
                     [SortingHelp] int,
                     [Realtime] int,
                     [MeansOfTransport] varchar(50)
                                     )
                    '''
create_table('linien',sql_command, server, db_name)


Connection with DESKTOP-JV1HTQR\SQLEXPRESS sucessfull!
Current Database: WienerLinienDB
DB Connection Status: True

Table 'linien' available in WienerLinienDB

DB-Connection with Server DESKTOP-JV1HTQR\SQLEXPRESS closed.
DB Connections Status: False


In [17]:
def fill_linien(sql_command, df, server, db_name):
    conn = db_connect(server=server, db_name=db_name)
    with conn:
        crs=conn.cursor()
        #try:
        for index, row in df.iterrows():
            crs.execute(sql_command, row['LineID'],
                                     row['LineText'],
                                     row['SortingHelp'],
                                     row['Realtime'],
                                     row['MeansOfTransport']
                                    )
        print('Table sucessfully filled!')
        #except:
        print('Not Sucessfull!')
    close_connection(server = server, conn=conn)

In [18]:
sql_command= '''INSERT INTO linien
           (
             [LineID],
             [LineText] ,
             [SortingHelp],
             [Realtime] ,
             [MeansOfTransport]
                                     )
     VALUES
           (?,
            ?,
            ?,
            ?,
            ?);
'''
fill_linien(sql_command, data['clean_linien'], server, db_name)


Connection with DESKTOP-JV1HTQR\SQLEXPRESS sucessfull!
Current Database: WienerLinienDB
DB Connection Status: True
Table sucessfully filled!
Not Sucessfull!

DB-Connection with Server DESKTOP-JV1HTQR\SQLEXPRESS closed.
DB Connections Status: False


### Create and fill haltestellen

In [19]:
# create table haltestellen
sql_command = '''IF NOT EXISTS(SELECT * FROM information_schema.tables
                 WHERE table_name='haltestellen' AND table_schema='dbo')
                 CREATE TABLE haltestellen (
                    [DIVA] int,
                    [PlatformText] varchar(50),
                    [Municipality] varchar(50),
                    [MunicipalityID] int,
                    [Longitude] numeric(18,6),
                    [Latitude] numeric(18,6)
                                     )
                    '''
create_table('haltestellen',sql_command, server, db_name)


Connection with DESKTOP-JV1HTQR\SQLEXPRESS sucessfull!
Current Database: WienerLinienDB
DB Connection Status: True

Table 'haltestellen' available in WienerLinienDB

DB-Connection with Server DESKTOP-JV1HTQR\SQLEXPRESS closed.
DB Connections Status: False


In [20]:
def fill_haltestellen(sql_command, df, server, db_name):
    conn = db_connect(server=server, db_name=db_name)
    with conn:
        crs=conn.cursor()
        #try:
        for index, row in df.iterrows():
            crs.execute(sql_command, row['DIVA'],
                                     row['PlatformText'],
                                     row['Municipality'],
                                     row['MunicipalityID'],
                                     row['Longitude'],
                                     row['Latitude']
                                    )
        print('Table sucessfully filled!')
        #except:
        print('Not Sucessfull!')
    close_connection(server = server, conn=conn)

In [21]:
sql_command= '''INSERT INTO [dbo].[haltestellen]
           (
            [DIVA],
            [PlatformText],
            [Municipality],
            [MunicipalityID],
            [Longitude],
            [Latitude]
                                     )
     VALUES
           (?,
            ?,
            ?,
            ?,
            ?,
            ?);
'''
fill_haltestellen(sql_command, data['clean_haltestellen'], server, db_name)


Connection with DESKTOP-JV1HTQR\SQLEXPRESS sucessfull!
Current Database: WienerLinienDB
DB Connection Status: True
Table sucessfully filled!
Not Sucessfull!

DB-Connection with Server DESKTOP-JV1HTQR\SQLEXPRESS closed.
DB Connections Status: False


### Create and fill fahrwegverlaeufe

In [22]:
# create table fahrwegverlaeufe
sql_command = '''IF NOT EXISTS(SELECT * FROM information_schema.tables
                 WHERE table_name='fahrwegverlaeufe' AND table_schema='dbo')
                 CREATE TABLE [fahrwegverlaeufe] (
                        [LineID] int,
                        [PatternID] int,
                        [StopSeqCount] int,
                        [StopID] int
                                                 )
                    '''
create_table('fahrwegverlaeufe',sql_command, server, db_name)


Connection with DESKTOP-JV1HTQR\SQLEXPRESS sucessfull!
Current Database: WienerLinienDB
DB Connection Status: True

Table 'fahrwegverlaeufe' available in WienerLinienDB

DB-Connection with Server DESKTOP-JV1HTQR\SQLEXPRESS closed.
DB Connections Status: False


In [23]:
def fill_fahrwegverlaeufe(sql_command, df, server, db_name):
    conn = db_connect(server=server, db_name=db_name)
    with conn:
        crs=conn.cursor()
        try:
            for index, row in df.iterrows():
                crs.execute(sql_command, int(row['LineID']),
                                         int(row['PatternID']),
                                         int(row['StopSeqCount']),
                                         int(row['StopID'])
                                        )
            print('Table sucessfully filled!')
        except:
            print('Not Sucessfull!')
    close_connection(server = server, conn=conn)

In [24]:
sql_command= '''INSERT INTO [dbo].[fahrwegverlaeufe]
           (
            [LineID],
            [PatternID],
            [StopSeqCount],
            [StopID]
                                     )
     VALUES
           (?,
            ?,
            ?,
            ?);
'''
fill_fahrwegverlaeufe(sql_command, data['clean_fahrwegverlaeufe'], server, db_name)


Connection with DESKTOP-JV1HTQR\SQLEXPRESS sucessfull!
Current Database: WienerLinienDB
DB Connection Status: True
Table sucessfully filled!

DB-Connection with Server DESKTOP-JV1HTQR\SQLEXPRESS closed.
DB Connections Status: False


### Create and fill gps-punkte

In [25]:
# create table fahrwegverlaeufe
sql_command = '''IF NOT EXISTS(SELECT * FROM information_schema.tables
                 WHERE table_name='gps_punkte' AND table_schema='dbo')
                 CREATE TABLE [gps_punkte] (
                    [FromStartStop] int,
                    [ToEndStop] int,
                    [TransportMode] int,
                    [SeqCount] int,
                    [Longitude] numeric(18,6),
                    [Latitude] numeric(18,6),
                    [DistanceFromStartPoint] int
)
                    '''
create_table('gps_punkte',sql_command, server, db_name)


Connection with DESKTOP-JV1HTQR\SQLEXPRESS sucessfull!
Current Database: WienerLinienDB
DB Connection Status: True

Table 'gps_punkte' available in WienerLinienDB

DB-Connection with Server DESKTOP-JV1HTQR\SQLEXPRESS closed.
DB Connections Status: False


In [26]:
def fill_gps_punkte(sql_command, df, server, db_name):
    conn = db_connect(server=server, db_name=db_name)
    with conn:
        crs=conn.cursor()
        try:
            for index, row in df.iterrows():
                crs.execute(sql_command, int(row['FromStartStop']),
                                         int(row['ToEndStop']),
                                         int(row['TransportMode']),
                                         int(row['SeqCount']),
                                         row['Longitude'],
                                         row['Latitude'],
                                         row['DistanceFromStartPoint'])
            print('Table sucessfully filled!')
        except:
            print('Not Sucessfull!')
    close_connection(server = server, conn=conn)

In [27]:
sql_command= '''INSERT INTO [dbo].[gps_punkte]
           (
            [FromStartStop],
            [ToEndStop],
            [TransportMode],
            [SeqCount],
            [Longitude],
            [Latitude],
            [DistanceFromStartPoint]
            )
     VALUES
           (?,
            ?,
            ?,
            ?,
            ?,
            ?,
            ?);
'''
fill_gps_punkte(sql_command, data['clean_gps-punkte'], server, db_name)


Connection with DESKTOP-JV1HTQR\SQLEXPRESS sucessfull!
Current Database: WienerLinienDB
DB Connection Status: True
Table sucessfully filled!

DB-Connection with Server DESKTOP-JV1HTQR\SQLEXPRESS closed.
DB Connections Status: False


### Create and fill steige

In [28]:
# create table fahrwegverlaeufe
sql_command = '''IF NOT EXISTS(SELECT * FROM information_schema.tables
                 WHERE table_name='steige' AND table_schema='dbo')
                 CREATE TABLE [steige] (
                    [StopID] int,
                    [Platform] varchar(50)
                                            )
                    '''
create_table('steige',sql_command, server, db_name)


Connection with DESKTOP-JV1HTQR\SQLEXPRESS sucessfull!
Current Database: WienerLinienDB
DB Connection Status: True

Table 'steige' available in WienerLinienDB

DB-Connection with Server DESKTOP-JV1HTQR\SQLEXPRESS closed.
DB Connections Status: False


In [29]:
def fill_steige(sql_command, df, server, db_name):
    conn = db_connect(server=server, db_name=db_name)
    with conn:
        crs=conn.cursor()
        try:
            for index, row in df.iterrows():
                crs.execute(sql_command, int(row['StopID']),
                                         row['Platform']
                                        )
            print('Table sucessfully filled!')
        except:
            print('Not Sucessfull!')
    close_connection(server = server, conn=conn)

In [30]:
sql_command= '''INSERT INTO [dbo].[steige]
           (
            [StopID],
            [Platform]
            )
     VALUES
           (?,
            ?);
'''
fill_steige(sql_command, data['clean_steige'], server, db_name)


Connection with DESKTOP-JV1HTQR\SQLEXPRESS sucessfull!
Current Database: WienerLinienDB
DB Connection Status: True
Table sucessfully filled!

DB-Connection with Server DESKTOP-JV1HTQR\SQLEXPRESS closed.
DB Connections Status: False


### Create stage_delay

In [31]:
# create table stage_delay
sql_command = '''IF NOT EXISTS(SELECT * FROM information_schema.tables
                 WHERE table_name='stage_delay' AND table_schema='dbo')
                 CREATE TABLE stage_delay (
                    [timestamp] datetime2,
                    [stop_name] varchar(100),
                    [line_name] varchar(50),
                    [lineID] varchar(50),
                    [towards] varchar(100),
                    [richtungsID] varchar(50),
                    [type] varchar(50),
                    [timePlanned] datetime2,
                    [timeReal] datetime2,
                    [DIVA] int,
                    [rbl] int,
                    [delay] numeric(18,0))
                '''

create_table('stage_delay', sql_command, server, db_name)


Connection with DESKTOP-JV1HTQR\SQLEXPRESS sucessfull!
Current Database: WienerLinienDB
DB Connection Status: True

Table 'stage_delay' available in WienerLinienDB

DB-Connection with Server DESKTOP-JV1HTQR\SQLEXPRESS closed.
DB Connections Status: False


### Create delay

In [32]:
# create table delay
sql_command = '''IF NOT EXISTS(SELECT * FROM information_schema.tables
                 WHERE table_name='delay' AND table_schema='dbo')
                 CREATE TABLE delay (
                    [timestamp] datetime2,
                    [stop_name] varchar(100),
                    [line_name] varchar(50),
                    [lineID] varchar(50),
                    [towards] varchar(100),
                    [richtungsID] varchar(50),
                    [type] varchar(50),
                    [timePlanned] datetime2,
                    [timeReal] datetime2,
                    [DIVA] int,
                    [rbl] int,
                    [delay] numeric(18,0))
                '''

create_table('delay', sql_command, server, db_name)


Connection with DESKTOP-JV1HTQR\SQLEXPRESS sucessfull!
Current Database: WienerLinienDB
DB Connection Status: True

Table 'delay' available in WienerLinienDB

DB-Connection with Server DESKTOP-JV1HTQR\SQLEXPRESS closed.
DB Connections Status: False


It looks like everything works now. 
# Next Steps
___
Now in the next step a script for extracting live data from the Wiener Linien API will be developed. This file's name is `API_Extraction`.